In [9]:
import pandas as pd
from utilitarios.geonames import get_geolocalizacao
#base = get_geolocalizacao(cod_municipio=1100015)
base = pd.DataFrame(get_geolocalizacao(cod_municipio=1100015))
base

,UF,COD_UF,MUNICIPIO,BAIRRO,LATITUDE,LONGITUDE
0,RO,24,1100015,POSTO FISCAL ROLIM DE MOURA,-13.08271,-62.27726
1,RO,24,1100015,ALTA FLORESTA D'OESTE,-11.96806,-61.95417
2,RO,24,1100015,CLIZA,-12.61667,-62.06667
3,RO,24,1100015,URUROSES,-12.86667,-62.2
4,RO,24,1100015,CONSUELO,-12.65,-62.13333
5,RO,24,1100015,ALTA FLORESTA D'OESTE,-11.92871,-61.99829


In [6]:
with open("auxiliares\MUNICIPIOS.CSV", "r", encoding="utf-8") as m:
    geo_municipios = m.readlines()[1:]
    m.close()
for municipio in geo_municipios:
    # usando os separadores para pegar o campo que quero
    partes_municipio = municipio.split(";")
    print(type(partes_municipio[0]))
    print(type(int(partes_municipio[0])))
    print(int(partes_municipio[0]))

    break

<class 'str'>
<class 'int'>
1100015


In [2]:
base

[{'UF': 'MG',
  'COD_UF': '15',
  'MUNICIPIO': '3106200',
  'BAIRRO': 'BELO HORIZONTE',
  'LATITUDE': '-19.92083',
  'LONGITUDE': '-43.93778'},
 {'UF': 'MG',
  'COD_UF': '15',
  'MUNICIPIO': '3106200',
  'BAIRRO': 'PAMPULHA',
  'LATITUDE': '-19.9',
  'LONGITUDE': '-43.93333'},
 {'UF': 'MG',
  'COD_UF': '15',
  'MUNICIPIO': '3106200',
  'BAIRRO': 'CONFINS',
  'LATITUDE': '-19.6325',
  'LONGITUDE': '-43.98278'},
 {'UF': 'MG',
  'COD_UF': '15',
  'MUNICIPIO': '3106200',
  'BAIRRO': 'OLHOS DE AGUA',
  'LATITUDE': '-20.0012',
  'LONGITUDE': '-43.9733'},
 {'UF': 'MG',
  'COD_UF': '15',
  'MUNICIPIO': '3106200',
  'BAIRRO': 'ANTONIO RIBEIRO DE ABREU',
  'LATITUDE': '-19.83404',
  'LONGITUDE': '-43.90532'},
 {'UF': 'MG',
  'COD_UF': '15',
  'MUNICIPIO': '3106200',
  'BAIRRO': 'VILA BETANIA',
  'LATITUDE': '-19.96391',
  'LONGITUDE': '-43.9975'},
 {'UF': 'MG',
  'COD_UF': '15',
  'MUNICIPIO': '3106200',
  'BAIRRO': 'VILA ENGENHO NOGUEIRA',
  'LATITUDE': '-19.88975',
  'LONGITUDE': '-43.98446'},

In [ ]:
import pandas as pd
import sqlalchemy
dados = pd.read_parquet('br_base\Restaurantes e similares.gzip')
dados.columns

In [ ]:
import duckdb

# Criando um objeto duckdb
duck = duckdb.connect()

# consulta
query = """
SELECT DATA_SITUACAO_CADASTRAL, 
COUNT(*) as TOTAL
FROM dados 
WHERE CNAE_DESCRICAO = 'RESTAURANTES E SIMILARES'
GROUP BY DATA_SITUACAO_CADASTRAL;
"""

# Exemplo de consulta SQL usando duckdb
resultado = duck.query(query).df()

resultado

In [ ]:
ultima_alteracao = duck.query("""
SELECT DATA_SITUACAO_CADASTRAL 
FROM dados
ORDER BY DATA_SITUACAO_CADASTRAL DESC
LIMIT 1;
""").fetchone()[0]

ultima_alteracao